In [1]:
from torch.utils.data.sampler import SubsetRandomSampler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tensorboard_logger import configure, log_value
from model import RecurrentAttention
from torchvision import transforms
from torchvision import datasets
import torch.nn.functional as F
import matplotlib.pyplot as plt
from utils import AverageMeter
from config import get_config
from utils import plot_images
from trainer import Trainer
from tqdm import tqdm
import pandas as pd
import torchvision
import numpy as np
import random
import pickle
import shutil
import torch
import time

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

from utils import *

In [2]:
q = 4
with open("../../pooling/data/migration_data.json") as m:
    mig_data = json.load(m)
m.close()
mig_data = pd.DataFrame.from_dict(mig_data, orient = 'index').reset_index()
mig_data.columns = ['muni_id', 'num_migrants']
mig_data['class'] = pd.qcut(mig_data['num_migrants'], q = q, labels = [i for i in range(q)])
mig_data.head()

,muni_id,num_migrants,class
0,484001001,42055.0,3
1,484001002,4017.0,3
2,484001003,11992.0,3
3,484001004,762.0,2
4,484001005,7551.0,3


In [3]:
image_names = get_png_names("../../attn/data/MEX/")
y_class, y_mig = get_ys(image_names, mig_data)

with open("../val_images.txt") as ims:
    val_names = ims.read().splitlines()
    
train_names = [i for i in image_names if i not in val_names]

val_indices = [image_names.index(i) for i in val_names]
train_indices = [image_names.index(i) for i in train_names]
train_num = int(len(train_indices))

In [4]:
# train_indices = train_indices[0:25]
# val_indices = val_indices[0:10]

In [5]:
# image_names[2].split("/")[5]

In [6]:
batch_size = 1
train = [(torchvision.transforms.functional.adjust_brightness(load_inputs(image_names[i]), brightness_factor = 2).squeeze(), y_mig[i], image_names[i].split("/")[5]) for i in train_indices]
val = [(torchvision.transforms.functional.adjust_brightness(load_inputs(image_names[i]), brightness_factor = 2).squeeze(), y_mig[i], image_names[i].split("/")[5]) for i in val_indices]
train_dl = torch.utils.data.DataLoader(train, batch_size = batch_size, shuffle = True)
val_dl = torch.utils.data.DataLoader(val, batch_size = batch_size, shuffle = True)

In [7]:
print("Num training: ", len(train_dl))
print("Num validation: ", len(val_dl))

Num training:  1446
Num validation:  620


In [8]:
# mu = torch.tensor([[-0.0070, -0.0172]])
# std = 0.4
# torch.distributions.Normal(mu, std).mean

In [9]:
# def get_dists(mu, std, n):
#     return torch.cat([torch.distributions.Normal(mu, std).rsample() for i in range(n)]).numpy().tolist()

# get_dists(mu, std, 5)    

In [10]:
# import json

In [11]:
# with open('test.json', "r") as f:
#     cur = json.load(f)
    
# cur[k] = 

In [12]:
# with open('test.json', "w") as f:
#     json.dump({1: get_dists(mu, std, 5)}, f)

In [13]:
# torch.distributions.Normal(mu, std).rsample()

In [14]:
# torch.distributions.Normal(mu, std).sample_n(5)

In [ ]:
config, unparsed = get_config()
trainer = Trainer(config, (train_dl, val_dl))
trainer.train()

0.2s - loss: 795.167 - mae: 795.163:   0%|          | 2/1446 [00:00<01:54, 12.57it/s]


[*] Train on 1446 samples, validate on 620 samples

Epoch: 1/200 - LR: 0.000300


481.0s - loss: 750.502 - mae: 727.010: 100%|██████████| 1446/1446 [08:00<00:00,  3.01it/s]    


train loss: 1874.693 - val loss: 1617.186- train mae: 1865.280 - val mae: 1601.236 [*]


0.2s - loss: 12701.476 - mae: 12694.146:   0%|          | 2/1446 [00:00<02:14, 10.72it/s]

...

  0%|          | 0/1446 [00:00<?, ?it/s]

train loss: 1798.809 - val loss: 1547.854- train mae: 1787.908 - val mae: 1539.483

Epoch: 11/200 - LR: 0.000300


  0%|          | 0/1446 [00:00<?, ?it/s]

train loss: 1800.774 - val loss: 1551.030- train mae: 1789.573 - val mae: 1542.110

Epoch: 12/200 - LR: 0.000300


480.8s - loss: 4155.514 - mae: 4154.651: 100%|██████████| 1446/1446 [08:00<00:00,  3.01it/s]  
